# **Introduction to CUDA Python with Numba**

Numba is a compiler for Python array and numerical functions that gives you the power to speed up your applications with high performance functions written directly in Python.

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest.

Numba supports CUDA GPU programming by directly compiling a restricted subset of Python code into CUDA kernels and device functions following the CUDA execution model. Kernels written in Numba appear to have direct access to NumPy arrays. NumPy arrays are transferred between the CPU and the GPU automatically.

[Documentation](https://numba.readthedocs.io/en/stable/cuda/index.html)

#### **Terminology**
* **host**: the CPU
* **device**: the GPU
* **host memory**: the system main memory
* **device memory**: onboard memory on a GPU card
* **kernels**: a GPU function launched by the host and executed on the device
* **device function**: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

Most CUDA programming facilities exposed by Numba map directly to the CUDA C language offered by NVidia. Therefore, it is recommended you read the official [CUDA C programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/).



# Import libraries

In [ ]:
import numpy as np
import math
import random
from decimal import Decimal

import numba
from numba import jit
from numba import vectorize 
from numba import guvectorize
from numba import cuda

Checking Numba version


In [ ]:
print(numba.__version__)

# Compiling Python code with @jit

The CUDA JIT is a low-level entry point to the CUDA features in Numba. It translates Python functions into [PTX](http://en.wikipedia.org/wiki/Parallel_Thread_Execution) code which execute on the CUDA hardware. The jit decorator is applied to Python functions written in our [Python dialect for CUDA](https://numba.pydata.org/numba-doc/0.13/CUDAPySpec.html). Numba interacts with the [CUDA Driver API](http://docs.nvidia.com/cuda/cuda-driver-api/index.html) to load the PTX onto the CUDA device and execute.

## Compiling functions on the CPU
Numba provides several utilities for code generation, but its central feature is the numba.jit() decorator with an explicit signature. Using Numba decorator @jit, which creates a normal function for execution on the CPU.

 Let’s start by peeking at the numba.jit string-doc:

In [ ]:
print(numba.jit.__doc__)

The recommended way to use the @jit decorator is to let Numba decide when and how to optimize. Let's make a compiled version of a function. In this mode, compilation will be deferred until the first function execution. Numba will infer the argument types at call time, and generate optimized code based on this information.

In [ ]:
nsamples = 2000

# JIT compile a python function conforming to the CUDA-Python specification. To define a CUDA kernel:
@jit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

Let's compare the speed between the compiled and uncompiled versions. **.py_func** attribute that can be used to access the original uncompiled Python function. 

In [ ]:
%timeit monte_carlo_pi.py_func(nsamples)

In [ ]:
%timeit monte_carlo_pi(nsamples)

The Numba-compiled version had a significant speed-up!

## Compilation options

A number of keyword-only arguments can be passed to the @jit decorator.

Numba has two compilation modes: **nopython mode** and **object mode**.

 ```nopython mode``` produces much faster code, but has limitations that can force Numba to fall back to the ```object mode```. To prevent Numba from falling back, and instead raise an error, pass ```nopython=True```.

Let's see nopython mode. The nopython mode will generate the best performance, but has limitations.

In [ ]:
#function without error

@jit("void(f4[:])",nopython=True)
def squared(a):
    squared_val = a*a


Types that can’t be inferred by the compiler in the nopython mode and it will generate an error.

In [ ]:
#function that contains a variable whose type can’t be inferred by the compiler
#nopython mode set to True

@jit("void(f4[:])",nopython=True)
def squared(a):
    squared_val = a*a
    val = Decimal(100)

If we don’t specify anything like in the function below, where nopython mode is not set to True, the compilation is falling back to object mode and produces a warning but not an error.

In [ ]:
# function that contains a variable whose type can’t be inferred by the compiler
# nopython mode not set to True
# compilation is falling back to object mode

@jit("void(f4[:])")
def squared(a):
    squared_val = a*a
    val = Decimal(100)

## Type inference

The objective of type inference is assigning a type to every single value in the function. The type of a value can either be:

* Implicit, in the case of providing an object that will provide its type. For e.g. in literals.
* Explicit, in the case of the programmer explicitly writing the type of a given value. For e.g. when a signature is given to numba.jit. That signature explicitly types the arguments.
* Inferred, when the type is deduced from an operation and the types of its operands. For e.g. inferring that the type of a + b, when a and b are of type int is going to be an int
Type inference is the process by which all the types that are neither implicit nor explicit are deduced.

In [ ]:
monte_carlo_pi.inspect_types()

Numba supports generating NumPy ufuncs and gufuncs. In NumPy there are universal functions(ufuncs) and generalized universal functions (gufuncs). 

# Creating Numpy universal functions

## Universal functions

Universal functions(ufuncs) are functions that broadcast an elementwise operation across input arrays of varying numbers of dimensions. Most NumPy functions are ufuncs, and Numba makes it easy to compile custom ufuncs using the @vectorize decorator.

In [ ]:
def cpu_sqrt(x):
  return math.sqrt(x)

In [ ]:
@vectorize
def cpu_numba_sqrt(x):
  return math.sqrt(x)

In [ ]:
@vectorize(['float32(float32)'], target='cuda')
def gpu_numba_sqrt(x):
    return math.sqrt(x)

In [ ]:
%timeit np.sqrt(25)

In [ ]:
%timeit cpu_sqrt(25)

In [ ]:
%timeit cpu_numba_sqrt(25)

In [ ]:
%timeit gpu_numba_sqrt(25)

Why did we see an increase in the processing time?

An overhead is introduced by Numba to each function call that is larger than the function call overhead of Python itself. 

Functions that are quick to compute are affected by this.

Since the function here is too simple to run on the GPU, we experience longer processing time due to data transfer operations taking place.

* input data is transferred to the GPU memory
* the square root is calculated on the GPU
* the resulting value is sent back to the host system

## Generalized ufuncs on the GPU

In generalized ufuncs (gufuncs), the calculation can deal with a sub-array of the input array, and return an array of different dimensions.

In [ ]:
#calculating the sum of elements in each row of the array

@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_sum(a, out):
    sum = 0
    for val in a: 
        sum += val
    out[0] = sum

In [ ]:
a = np.arange(50).reshape(10, 5).astype(np.float32)
a

In [ ]:
calc_sum(a)

# Writing Device Functions

## Device functions

Uptil now we were compiling all our code in single functions but it’s possible to modularize and write clean code with the help of helper functions for the GPU. These are called **device** functions.

`add_values:` device function to sum up the elements of the array using the numba.cuda.jit decorator

`calc_sum:` gufunc to make use of the add_values device function

`calc_avg:` gufunc to calculate the average

In [ ]:
@cuda.jit(device=True)
def add_values(a): 
  sum = 0
  for val in a: 
    sum += val
  return sum

@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_sum(a, out):
    out[0] = add_values(a)

@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_avg(a, out):
    out[0] = add_values(a)/len(a)

In [ ]:
calc_sum(a)

In [ ]:
calc_avg(a)